In [44]:
import pandas as pd
import numpy as np

def parse_HMM(file): # with pseudocount sigma
    lines = f.readlines()
    text = lines[0].strip()
    # if error: check if tab or space in this input line
#     (theta, sigma) = (float(i) for i in lines[2].strip().split('\t'))
    (theta, sigma) = (float(i) for i in lines[2].strip().split(' '))
    alpha = lines[4].strip().split()
    alpha = dict(zip(alpha, range(len(alpha))))
    alignment = [line.strip() for line in lines[6:]]
    print("Text: ", text)
    print("Alignment:")
    for a in alignment:
        print('\t', a)
    print("Alpha:", alpha)
    print("Theta:", theta)
    print("Sigma:", sigma)
    return text, alignment, alpha, theta, sigma

### algorithm for building profile HMM from an alignment|theta
def profileHMM(alignment, alpha, theta):

    def get_seed_alignment(alignment, theta, alphabet):
        """returns seed alignment columns for HMM"""
        k = len(alignment[0]); a = len(alphabet.keys())
        freq = np.zeros(shape=(a+1, k)) 
        for seq in alignment:
            for i in range(k):
                if seq[i] == '-':
                    freq[a][i] += 1
                else:
                    freq[alphabet[seq[i]]][i] +=1
        n = len(alignment)
        seed = [x/n < theta for x in freq[a]]
        return seed

    def state_transition(T, prev, kind, S):
        """ find next Match, Del, Ins, or End state in [states] """
        x=0
        if S[prev][0] == 'M':
            x=1
        for nxt in range(prev+1+x, len(T)):
            if S[nxt][0] == kind[0]:
                T[prev][nxt] += 1
                return T, nxt
            
    def normalize_matrices(T,E):
        """Normalize Transmission and Emission frequencies to sum of row for state"""
        for state in range(len(S)):
            if sum(T[state]) > 0:
                T[state] =  T[state]/sum(T[state])
            if sum(E[state]) >0:
                E[state] =  E[state]/sum(E[state])
        return T, E
    
    def pseudo_normalize(T, E):
        """adds pseudocounts to HMM matrices"""
        # add pseudos and normalize E
        for i in range(1, len(S)-1):
            if S[i][0] != 'D':
                E[i] += 0.01
                E[i] = E[i]/sum(E[i])
        # add pseudos and normalize T
        for i in range(len(S)-1):
            start = 3*((i+1)//3)+1
            end = start+3
            for j in range(start, end):
                if j < len(S):
                    T[i][j] += 0.01
            T[i] = T[i]/sum(T[i])
        return T, E
    
    ### Walk each sequence to count transitions, emissions
    seed = get_seed_alignment(alignment, theta, alpha)
    n = len(alignment); k = len(alignment[0])
    S = ['S','I0']+list(c+str(n) for n in range(1,sum(seed)+1) for c in "MDI")+['E']
    E = np.zeros(shape=(len(S), len(alpha.keys())))
    T = np.zeros(shape=(len(S), len(S)))

    for seq in alignment:
        # 'state' is the hidden state col/row; i is pos in alignment
        state = 0; i = 0
        while i < k:
            # seed: either match or del as hidden state
            if seed[i]:
                if seq[i] in alpha:
                    T, state = state_transition(T, state, 'Match', S)
                    E[state][alpha[seq[i]]] +=1
                else:
                    T, state = state_transition(T, state, 'Deletion', S)
            # not seed: either insert or nothing
            else:
                # count any emissions before next seed column
                emits = []
                while not seed[i]:
                    if seq[i] in alpha:
                        emits.append(seq[i])
                    i += 1
                    if i == k: 
                        break  # hit end of sequence
                i-=1                
                if len(emits)>0:
                    # count the transition(state, insertion)
                    T, state = state_transition(T, state, 'Insert', S)
                    # get all emissions(state) in insertion
                    for symbol in emits:
                        E[state][alpha[symbol]] += 1
                    # count all symbols as cyclic transition t(ins_x, ins_x)
                    if len(emits) >1:
                        T[state][state] += len(emits)-1
                # else do nothing, just gaps in align
            i += 1
        # from last state to 'End'    
        T, state = state_transition(T, state, 'End', S)

    T,E = normalize_matrices(T,E)
    T, E = pseudo_normalize(T, E)
    return S, T, E, seed



def Viterbi_alignment(T,E,S,alpha,text):
    """Aligns text to HMM using Viterbi algorithm for optimal hidden path"""
    
    # initialize the Viterbi graph (|states|*|text|)
    Viterbi = np.ones(shape=(len(S)-1, len(text)+1))*-float('inf')
    pointers = [[False]+[False for t in text] for s in S[:-1]] 
    Viterbi[0][0] = np.log(1)
    pointers[0][0] = 'root'
    offsets = {'M':(3,1),'D':(4,0), 'I':(2,1)}

    # init 1st column all del states (not match or ins)
    prev = (0)
    for state in range(len(S)):
        if S[state][0]=='D':
            Viterbi[state][0] = Viterbi[prev][0] + np.log(T[prev][state])
            pointers[state][0] = (prev,0)
            prev=state

    for column in range(1, len(text)+1):
        symbol = alpha.index(text[column-1])
        for state in range(1, len(S[:-1])):
            (x,y) = offsets[S[state][0]]
            best_score = -float('inf')
            if S[state][0]=='D':
                p_emit = 1
            else:
                p_emit = E[state][symbol]
#             print('\nCurrent cell', S[state],(state, column), "; symbol:", text[column-1],
#                   "p_emit =", np.around(p_emit,4))
            for prev in range(state-x, state-x+3):
                if not prev<0:
                    p_prev = Viterbi[prev][column-y]
                    p_trans = T[prev][state]
                    p_edge = p_prev + np.log(p_trans)
#                     print("\tedge:",S[prev], (prev, column-y), '–>',  S[state], (state, column),)
#                     print("\t\tViterbi: {}, Transition: {};;; Edge -> {}".format(np.around(p_prev,4),
#                                                                                  np.around(p_trans,4),
#                                                                                  np.around(p_edge,4)))
                    if p_edge > best_score:
                        best_score = p_edge
                        Viterbi[state][column] = p_edge + np.log(p_emit)
                        pointers[state][column] = (prev, column-y)
    #                     print("\t\t\t**best edge:", (prev, column-y), S[prev])

    print(pd.DataFrame(np.around(Viterbi,3), columns=['start']+list(text), index=S[:-1]))
    print(pd.DataFrame(pointers, columns=['start']+list(text), index=S[:-1]))                
    return Viterbi, pointers

def backtrack(Viterbi, pointers):
    ### walk back pointers:
    # start backtrack from greatest probability in last column of viterbi
    score = -float('inf')
    for state in range(len(S)-4,len(S)-1):
        if Viterbi[state][len(text)] > score:
            last = state
            score = Viterbi[state][len(text)]
    path = ['']
    print("Best Score at end of Path:", score, "at state:", S[last])

    # backtrack to recreate max likelihood hidden_path in reverse
    x,y = last, len(text)
    while path:
        path.append(S[x])    
        (x,y) = pointers[x][y]
        if (x,y) ==(0,0):
            path = ' '.join(path[-1:0:-1])
            break

    print('\n\nPATH:\n',path)
    return path

In [45]:
### Sample data:
with open('/Users/jasonmoggridge/Dropbox/Rosalind/Coursera_textbook_track/Course6/data/ba10g.sample.txt') as f:
    text, alignment, alpha, theta, sigma = parse_HMM(f)

S, T, E, seed = profileHMM(alignment, alpha, theta)
alpha = list(alpha.keys())
print(pd.DataFrame(np.around(T,3), index = S, columns=S))
print(pd.DataFrame(np.around(E,3), index = S, columns=alpha))
V, P = Viterbi_alignment(T,E,S, alpha, text)
path = backtrack(V,P)


Text:  AEFDFDC
Alignment:
	 ACDEFACADF
	 AFDA---CCF
	 A--EFD-FDC
	 ACAEF--A-C
	 ADDEFAAADF
Alpha: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5}
Theta: 0.4
Sigma: 0.01
      S     I0     M1     D1     I1     M2     D2     I2     M3     D3  ...  \
S   0.0  0.010  0.981  0.010  0.000  0.000  0.000  0.000  0.000  0.000  ...   
I0  0.0  0.333  0.333  0.333  0.000  0.000  0.000  0.000  0.000  0.000  ...   
M1  0.0  0.000  0.000  0.000  0.010  0.786  0.204  0.000  0.000  0.000  ...   
D1  0.0  0.000  0.000  0.000  0.333  0.333  0.333  0.000  0.000  0.000  ...   
I1  0.0  0.000  0.000  0.000  0.333  0.333  0.333  0.000  0.000  0.000  ...   
M2  0.0  0.000  0.000  0.000  0.000  0.000  0.000  0.010  0.981  0.010  ...   
D2  0.0  0.000  0.000  0.000  0.000  0.000  0.000  0.010  0.010  0.981  ...   
I2  0.0  0.000  0.000  0.000  0.000  0.000  0.000  0.333  0.333  0.333  ...   
M3  0.0  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  ...   
D3  0.0  0.000  0.000  0.000  0.000  0.0

#### Getting the right answer on the sample dataset now! yay! going to call it quits for tonight

In [48]:
### extra data:
with open('/Users/jasonmoggridge/Dropbox/Rosalind/Coursera_textbook_track/Course6/data/ba10g.extra.txt') as f:
    text, alignment, alpha, theta, sigma = parse_HMM(f)


S, T, E, seed = profileHMM(alignment, alpha, theta)
alpha = list(alpha.keys())
print(pd.DataFrame(np.around(T,3), index = S, columns=S))
print(pd.DataFrame(np.around(E,3), index = S, columns=alpha))
V, P = Viterbi_alignment(T,E,S,alpha, text)
path = backtrack(V,P)


Text:  EEBEABDCEEABCCCEEBDEDCADEDACCDCBBEECDBDACABDADCBE
Alignment:
	 EEBBA--C-DBAA-AECD--BDB---CC-DDCBBCEDE-EBB-DAEE-C
	 EEEEABBCEABBCDEE-DAEBDBAEDC-BDBCB--C-B-BCA-DAEECA
	 --CEB-ACCDEACEEEEDBEED-ADBCCDAC--BC--BDBCAEDAEECC
	 A--AABDCE-A-CD-ECD-EBBA-EDC-DACCBBCCD-D-BA-DAAEBC
	 EECEAB--EDDACCE-CD-E--B-EDCCD-CCBBCCD-DBBA--AE-CA
	 E-CDA-DCECAAECB-CDCEB-B-BDCCD---B--CD-DBCDBDAEB-C
	 EBCEAEDC-DABC--A-DCEDDBAED-CD-CCBBCCEBDB--BEA-EEC
	 AC-E-BDCEDAADDEECDEEB-CAEDC-DD-CBBCCD-DBCABDAEECC
	 EECBABDCEDEAEC-DCDC-BDBDEDA-D-AD-A-EABEB--BDA-ECC
Alpha: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
Theta: 0.359
Sigma: 0.01
       S     I0     M1     D1     I1     M2     D2   I2   M3   D3  ...  M42  \
S    0.0  0.010  0.873  0.118  0.000  0.000  0.000  0.0  0.0  0.0  ...  0.0   
I0   0.0  0.333  0.333  0.333  0.000  0.000  0.000  0.0  0.0  0.0  ...  0.0   
M1   0.0  0.000  0.000  0.000  0.010  0.738  0.252  0.0  0.0  0.0  ...  0.0   
D1   0.0  0.000  0.000  0.000  0.010  0.010  0.981  0.0  0.0  0.0  ...  0.0

In [49]:
path = path.split(' ')


In [50]:
len(path)

55

In [51]:
sol = 'M1 M2 M3 M4 M5 M6 M7 M8 M9 D10 M11 M12 I12 I12 M13 M14 M15 M16 M17 M18 D19 M20 M21 M22 M23 I23 M24 M25 M26 I26 I26 M27 D28 M29 M30 M31 I31 I31 D32 M33 M34 I34 M35 M36 M37 M38 I38 M39 M40 M41 M42 M43 M44'.split(' ')

In [52]:
len(sol)

53

In [53]:
sum(1 if s[0] in ('M','I') else 0 for s in path), sum(1 if s[0] in ('D') else 0 for s in path)

(49, 6)

In [54]:
sum(1 if s[0] in ('M','I') else 0 for s in sol), sum(1 if s[0] in ('D') else 0 for s in sol)

(49, 4)

In [55]:
v = pd.DataFrame(np.around(V,3), columns=['start']+list(text), index=S[:-1])
p = pd.DataFrame(P, columns=['start']+list(text), index=S[:-1])     
    

In [56]:
v[-10:][-10:]

,start,E,E,B,E,A,B,D,C,E,...,A,C,A,B,D,A,D,C,B,E
I41,-inf,-122.067,-114.436,-107.286,-99.583,-94.569,-90.384,-89.398,-89.569,-87.557,...,-80.458,-76.273,-78.981,-81.689,-84.397,-86.142,-81.957,-84.665,-87.373,-90.081
M42,-inf,-116.013,-108.383,-103.112,-93.530,-92.999,-86.211,-87.828,-87.999,-81.503,...,-78.887,-74.702,-78.287,-80.032,-85.605,-84.571,-80.386,-83.971,-86.345,-87.172
D42,-120.458,-112.827,-105.676,-97.974,-92.960,-88.774,-87.789,-87.960,-85.947,-81.065,...,-74.663,-75.369,-79.717,-82.787,-84.532,-80.347,-81.053,-85.763,-88.471,-89.081
I42,-inf,-126.702,-119.071,-111.920,-104.218,-99.204,-95.019,-92.455,-94.072,-92.191,...,-83.242,-80.907,-80.946,-83.654,-86.276,-88.984,-86.591,-86.630,-89.338,-92.046
M43,-inf,-122.404,-114.774,-107.623,-99.921,-97.633,-90.721,-91.169,-88.351,-87.894,...,-83.522,-75.054,-79.660,-80.518,-84.990,-89.206,-85.021,-81.061,-86.202,-88.576
D43,-125.092,-117.390,-109.759,-102.609,-94.906,-93.409,-87.587,-89.204,-89.375,-82.880,...,-79.298,-76.079,-79.664,-81.409,-86.982,-84.982,-81.763,-85.348,-87.721,-88.549
I43,-inf,-131.336,-123.634,-116.004,-108.853,-101.151,-99.653,-93.831,-95.448,-94.595,...,-81.209,-83.917,-81.298,-84.007,-86.715,-89.423,-91.226,-88.007,-87.306,-90.014
M44,-inf,-129.766,-122.064,-114.433,-107.282,-96.435,-98.083,-92.261,-89.511,-93.024,...,-81.109,-79.605,-76.583,-84.334,-85.192,-86.519,-89.655,-82.070,-85.735,-90.876
D44,-129.727,-122.025,-114.394,-107.243,-99.541,-98.044,-92.222,-93.839,-92.985,-87.514,...,-82.308,-79.689,-82.397,-85.105,-87.813,-89.617,-86.397,-85.696,-88.404,-91.112
I44,-inf,-132.030,-124.327,-116.697,-109.546,-101.844,-100.347,-94.525,-96.142,-95.288,...,-80.686,-82.989,-81.992,-82.817,-85.120,-87.422,-89.725,-88.700,-87.999,-90.301


In [57]:
p[-10:][-10:]

,start,E,E,B,E,A,B,D,C,E,...,A,C,A,B,D,A,D,C,B,E
I41,False,"(123, 0)","(123, 1)","(123, 2)","(123, 3)","(123, 4)","(123, 5)","(123, 6)","(123, 7)","(123, 8)",...,"(123, 39)","(123, 40)","(124, 41)","(124, 42)","(124, 43)","(123, 44)","(123, 45)","(124, 46)","(124, 47)","(124, 48)"
M42,False,"(123, 0)","(123, 1)","(123, 2)","(123, 3)","(123, 4)","(123, 5)","(123, 6)","(123, 7)","(123, 8)",...,"(123, 39)","(123, 40)","(122, 41)","(122, 42)","(123, 43)","(123, 44)","(123, 45)","(122, 46)","(123, 47)","(123, 48)"
D42,"(123, 0)","(123, 1)","(123, 2)","(123, 3)","(123, 4)","(123, 5)","(123, 6)","(123, 7)","(123, 8)","(123, 9)",...,"(123, 40)","(122, 41)","(122, 42)","(124, 43)","(123, 44)","(123, 45)","(122, 46)","(124, 47)","(124, 48)","(122, 49)"
I42,False,"(126, 0)","(126, 1)","(126, 2)","(126, 3)","(126, 4)","(126, 5)","(125, 6)","(125, 7)","(126, 8)",...,"(127, 39)","(126, 40)","(125, 41)","(127, 42)","(125, 43)","(127, 44)","(126, 45)","(125, 46)","(127, 47)","(127, 48)"
M43,False,"(126, 0)","(126, 1)","(126, 2)","(126, 3)","(126, 4)","(126, 5)","(125, 6)","(126, 7)","(126, 8)",...,"(126, 39)","(126, 40)","(125, 41)","(125, 42)","(125, 43)","(126, 44)","(126, 45)","(125, 46)","(125, 47)","(125, 48)"
D43,"(126, 0)","(125, 1)","(125, 2)","(126, 3)","(125, 4)","(126, 5)","(125, 6)","(125, 7)","(125, 8)","(125, 9)",...,"(126, 40)","(125, 41)","(125, 42)","(125, 43)","(125, 44)","(126, 45)","(125, 46)","(125, 47)","(125, 48)","(125, 49)"
I43,False,"(129, 0)","(129, 1)","(129, 2)","(129, 3)","(129, 4)","(129, 5)","(129, 6)","(129, 7)","(128, 8)",...,"(130, 39)","(130, 40)","(128, 41)","(130, 42)","(130, 43)","(130, 44)","(129, 45)","(129, 46)","(128, 47)","(130, 48)"
M44,False,"(129, 0)","(129, 1)","(129, 2)","(129, 3)","(129, 4)","(129, 5)","(129, 6)","(129, 7)","(128, 8)",...,"(130, 39)","(129, 40)","(128, 41)","(128, 42)","(128, 43)","(128, 44)","(129, 45)","(129, 46)","(128, 47)","(128, 48)"
D44,"(129, 0)","(129, 1)","(129, 2)","(129, 3)","(129, 4)","(129, 5)","(129, 6)","(129, 7)","(128, 8)","(129, 9)",...,"(130, 40)","(128, 41)","(130, 42)","(130, 43)","(130, 44)","(129, 45)","(129, 46)","(128, 47)","(130, 48)","(130, 49)"
I44,False,"(132, 0)","(132, 1)","(132, 2)","(132, 3)","(132, 4)","(132, 5)","(132, 6)","(132, 7)","(132, 8)",...,"(133, 39)","(133, 40)","(132, 41)","(131, 42)","(133, 43)","(133, 44)","(133, 45)","(132, 46)","(132, 47)","(133, 48)"


In [58]:
S[128]

'M43'

In [59]:
pwd

'/Users/jasonmoggridge/Dropbox/Rosalind/Coursera_textbook_track/Course6/code'

In [62]:
### stepik test data:
with open('/Users/jasonmoggridge/Dropbox/Rosalind/Coursera_textbook_track/Course6/data/dataset_26259_14.txt') as f:
    text, alignment, alpha, theta, sigma = parse_HMM(f)


S, T, E, seed = profileHMM(alignment, alpha, theta)
alpha = list(alpha.keys())
print(pd.DataFrame(np.around(T,3), index = S, columns=S))
print(pd.DataFrame(np.around(E,3), index = S, columns=alpha))
V, P = Viterbi_alignment(T,E,S,alpha,text)
path = backtrack(V,P)


Text:  ADDBACBDDDBACEABEEBDBEEDAECEAEBDCEADEEBCEADBDCABD
Alignment:
	 ADD-AEBCDEBECE-BE-BA-EEEAB-ECEBDCEABEE-B--DBEEB-D
	 A--EAEB-DEBEC-DB-EBCBEAAAB-EADBDC-AD-EDA-A-B-ABB-
	 -DAD-EB-DEBECBDBD-B--EDEABBE-ABD-EADEEBC-ADEDCBBD
	 CADCAE-D-E-E--DBEEBA-EEEC-BEADBCCEDABCBC-ADBD--BD
	 ADDDA-BED-BE-E-BEEB-BEEDA--EBDB-CEA-ECDCCA-BDABBB
	 ADDDAEDDD-BD-EDBEE-DD-E-D-B-ADBD-EB--EBCCE-BDABBD
Alpha: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
Theta: 0.327
Sigma: 0.01
       S     I0     M1     D1     I1     M2     D2   I2   M3   D3  ...  M32  \
S    0.0  0.010  0.819  0.172  0.000  0.000  0.000  0.0  0.0  0.0  ...  0.0   
I0   0.0  0.333  0.333  0.333  0.000  0.000  0.000  0.0  0.0  0.0  ...  0.0   
M1   0.0  0.000  0.000  0.000  0.010  0.786  0.204  0.0  0.0  0.0  ...  0.0   
D1   0.0  0.000  0.000  0.000  0.010  0.981  0.010  0.0  0.0  0.0  ...  0.0   
I1   0.0  0.000  0.000  0.000  0.333  0.333  0.333  0.0  0.0  0.0  ...  0.0   
..   ...    ...    ...    ...    ...    ...    ...  ...  ...  ...  ...  .

#### correct on the stepik dataset, now try rosalind

In [64]:
### rosalind test data:
with open('/Users/jasonmoggridge/Dropbox/Rosalind/Coursera_textbook_track/Course6/data/rosalind_ba10g.txt') as f:
    text, alignment, alpha, theta, sigma = parse_HMM(f)


S, T, E, seed = profileHMM(alignment, alpha, theta)
alpha = list(alpha.keys())
print(pd.DataFrame(np.around(T,3), index = S, columns=S))
print(pd.DataFrame(np.around(E,3), index = S, columns=alpha))
V, P = Viterbi_alignment(T,E,S,alpha, text)
path = backtrack(V,P)


Text:  EACCEECBECCADDEBABEEBEABDCDEEBECEBECBBDCDCDDCDCAA
Alignment:
	 EACCEEB-A-EAABEBA--ECCDBD-DCACE--BACAB-CB-DABA-D-
	 EAC--EBCE--A--EB-DEBCCDB--CCACEBE--DABCADCCABA-AC
	 EACCEEBBECEAABEBCEECCC-C-DDC-EECEBBAABACDC-ABAC-C
	 EACC-BB-E--A--EBA-EE--D-DCDCCCDB-BD-ABECDCD-BAEA-
	 -ACC-E-BEC---B-BA---CED-DCDCBCECEBC-ABACD-D-BA--C
	 CACEEE---CEA-BE--DEECBDBD-DCA--B----DB-CDCEBBA-AC
	 EACD--BDEC-AAB-BADEE-CD-BCDCAC-DABDCABA-ECDC-AEAE
	 EACC--BBECEAABDBADEECCDB-CADACB-E--CAB-C--DAB-BAC
Alpha: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
Theta: 0.297
Sigma: 0.01
       S     I0     M1     D1     I1     M2     D2   I2   M3   D3  ...  M31  \
S    0.0  0.010  0.859  0.131  0.000  0.000  0.000  0.0  0.0  0.0  ...  0.0   
I0   0.0  0.333  0.333  0.333  0.000  0.000  0.000  0.0  0.0  0.0  ...  0.0   
M1   0.0  0.000  0.000  0.000  0.010  0.981  0.010  0.0  0.0  0.0  ...  0.0   
D1   0.0  0.000  0.000  0.000  0.010  0.981  0.010  0.0  0.0  0.0  ...  0.0   
I1   0.0  0.000  0.000  0.000  0.333  0.333  0.3